In [28]:
# Dataset was taken form Mercari Price Suggestion Challenge
#   https://www.kaggle.com/c/mercari-price-suggestion-challenge
#
#  Second feature engineering: item discription detalisation for dummy variables


#import libs for data manipulation

import pandas as pd
import numpy as np

import xgboost as xgb


train = pd.read_csv('C:/temp/train.tsv', sep='\t' ,nrows= 1000)
#print (train.head())

In [29]:
#create dummy varibles for brand
train2=pd.DataFrame(train['brand_name'].str.replace(' ','').replace('.','').str.lower())
#print(train2)
dum=pd.get_dummies(train2['brand_name'])
#print(dum)

In [30]:
#load name words for dummy variables
df1=pd.read_csv(r'c:\Temp\dum.txt',delimiter=' ').reset_index()
df1['word']=df1['word'].str.replace('[','').str.replace(']','').str.replace("'","")
#print (df1)

In [31]:
#load discription words for dummy variables
df2=pd.read_csv(r'c:\Temp\dum1.txt',delimiter=' ').reset_index()
df2['word']=df2['word'].str.replace('[','').str.replace(']','').str.replace("'","")
#print (df2)

In [32]:
dumf=pd.concat([df1, df2], join='inner', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
dumf['i']=1
dump=pd.pivot_table(dumf, index=['word'],values=['i'],aggfunc='count').reset_index()
dumpf=dump.loc[dump['word']!='word'].drop(['i'],axis=1)
#print(dumpf)

In [33]:
#make dummy variables for name and item discription
for i in range(len(dumpf)):
    train['W'+dumpf[i:i+1].to_string(header= False,
                                 index= False)]=np.where((train['name'].str.lower()).str\
                                                         .find(dumpf[i:i+1].to_string(header= False, index= False))>-1,4,0)\
    +np.where((train['item_description'].str.lower()).str.find(dumpf[i:i+1].to_string(header= False, index= False))>-1,2,0)
print(train.columns.values)

['train_id' 'name' 'item_condition_id' 'category_name' 'brand_name' 'price'
 'shipping' 'item_description' 'W10x13' 'W14k' 'W6/6s' 'Wair' 'Wauthentic'
 'Wbag,' 'Wbeats' 'Wbelly' 'Wburberry' 'Wburch' 'Wcamera' 'Wcarters'
 'Wchanel' 'Wchoker' 'Wdecal' 'Wdust' 'Welf' 'Weyelashes' 'Weyeliner'
 'Wfidget' 'Wflowy' 'Wforever' 'Wgucci' 'Wipad' 'Wjordan' 'Wkate' 'Wkendra'
 'Wkors' 'Wlanyard' 'Wlaptop' 'Wlashes' 'Wlouis' 'Wlv' 'Wmascara'
 'Wmaybelline' 'Wmichael' 'Wmk' 'Wnail' 'Wnose' 'Wnyx' 'Wpatch' 'Wpoly'
 'Wpolymailers' 'Wpopsocket' 'Wprotector' 'Wretro' 'Wrue' 'Wsample'
 'Wsarah' 'Wsatchel' 'Wscott' 'Wsilicone' 'Wslime' 'Wspade' 'Wspinner'
 'Wsticker' 'Wstickers' 'Wthin' 'Wtiffany' 'Wtory' 'Wugg' 'Wvuitton']


In [34]:
#add dymmy for brand variables to main dataframe
train3 = pd.concat([train, dum], axis=1)
#print (train3)

In [35]:
#split our sample for train and test subsample
from sklearn.model_selection import train_test_split
X = train3.drop(['train_id','price','name','category_name','brand_name','item_description'], axis=1)
y = train3['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [36]:
#put in matrix our sample
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [18]:
#print(X.columns.values)

In [56]:
#set paramtres for model
param = {
    'max_depth': 30,  # the maximum depth of each tree
    'eta': 0.1,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 1000
}  # the number of classes that exist in this datset
num_round = 50  # the number of training iterations


In [57]:
#train model
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)

In [58]:
#calculate precision score
from sklearn.metrics import precision_score
best_preds = np.asarray([np.argmax(line) for line in preds])
print ("Numpy array precision:", precision_score(y_test, best_preds, average='macro'))

Numpy array precision: 0.002938045561


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
bst.dump_model('dump.raw.txt')